In [1]:
import os
import openai

proxy = 'http://dell-1.star:7890' # 3090 docker
# proxy = 'http://127.0.0.1:7890' # clash
# proxy = 'http://127.0.0.1:1080' # naiveproxy

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy
openai.api_key_path = ".openai-key2"

In [9]:
import json
with open("template/question_augmentation.json", "r") as f:
    template = json.load(f)


AUGMENT_PROMPT = """根据以下示例问题和答案，改写给定问题，不需要改写答案，要求改写后的问题与原问题的意思相同，且改写后的问题与给定答案匹配，但形式与原问题不同。
问题：
{question_example}
答案：
{answer_example}
改写后的问题：
{output_example}

问题：
{question}
答案：
{answer}
改写后的问题：
1.
"""


In [3]:
import logging
import random

def aug_questions_by_chat(row, max_tokens=1000):
    example = random.choice(template)
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": AUGMENT_PROMPT.format(
                        question_example=example['question'],
                        answer_example=example['answer'],
                        output_example=example['output'],
                        question=row['question'],
                        answer=row['answer'])
                }
            ],
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response.choices[0].message.content
    except Exception as e:
        logging.error(e)
        return ""


In [10]:
import pandas
from tqdm import tqdm
import json

tqdm.pandas()

filename = "template/self_cognition_seed.json"
with open(filename, "r") as f:
    data = json.load(f)
df = pandas.DataFrame(data)

df["aug_questions"] = df.progress_apply(aug_questions_by_chat, axis=1)

100%|██████████| 32/32 [01:22<00:00,  2.59s/it]


In [11]:
df["aug_questions"] = "1." + df.aug_questions
df.to_csv("data/self_cognition_aug.csv", index=False)

In [12]:
# split aug_questions
import pandas as pd
import re 

aug_df = pd.read_csv("data/self_cognition_aug.csv")
question_list, answer_list = [], []
for idx, row in aug_df.iterrows():
    aug_questions = row.aug_questions
    aug_questions = re.split(r"\d+\.", aug_questions)
    aug_questions = [q.strip() for q in aug_questions if q.strip()]
    question_list.extend(aug_questions)
    answer_list.extend([row.answer] * len(aug_questions))

new_df = pd.DataFrame({
    "question": question_list,
    "answer": answer_list
})

# new_df.to_csv("data/self_cognition.csv", index=False)
# new_df.to_json("data/self_cognition.jsonl", 
#                orient="records", force_ascii=False, lines=True)

In [15]:
import json

with open("template/self_cognition_seed.json", "r") as f:
    data = json.load(f)
raw_df = pandas.DataFrame(data)
# raw_df.to_json("output/self_cognition_seed.jsonl", force_ascii=False, lines=True,
#                orient="records")

new_df = pd.concat([raw_df, new_df], axis=0)
new_df.to_json("data/self_cognition.jsonl", force_ascii=False, lines=True,
                orient="records")

# After manual filtering, substitute key words

In [5]:
sub_rules = {
    "[OTHER_LLM_NAME]": ["ChatGPT", "Claude", "New Bing", "GPT4", 
                         "PaLM", "LLaMA", "Alpaca", "BLOOM", "OPT",
                         "Sparrow", "盘古α"],
    "[OTHER_GROUP_NAME]": ["OpenAI", "Microsoft", "Google", "Facebook", 
                           "腾讯", "华为", "字节跳动", "阿里巴巴", "百度", 
                           "智源", "商汤", "搜狗", "北大", "中科院", "人大"]
}

with open("data/self_cognition_filtered.jsonl", "r") as f:
    data = f.readlines()

with open("output/self_cognition/self_cognition.jsonl", "w") as f:
    for line in data:
        found = False
        for k, v in sub_rules.items():
            if k in line:
                found = True
                for vv in v:
                    new_line = line.replace(k, vv)
                    f.write(new_line)
        if not found:
            f.write(line)